In [1]:
from env import load_uavs, load_tasks, initialize_targets
from runEnv import UAVEnv

In [2]:
# 设置数据目录
uav_csv = "data/uav.csv"
task_csv = "data/task.csv"
uavs = load_uavs(uav_csv)
tasks = load_tasks(task_csv)
targets = initialize_targets(tasks)

# 俩个模式：DQN和Attention
dqn_mode = "dqn"
attention_mode = "attention"
env = UAVEnv(uavs, targets, tasks, mode=dqn_mode)

In [ ]:
"""
运行 DQN 优化
"""
# from train.dqn_train import train_dqn
# model = train_dqn(env)

In [4]:
"""
运行 AttentionDQN 优化
"""
# from train.attention_train import train_attention_dqn
# model = train_attention_dqn(env)

'\n运行 AttentionDQN 优化\n'

In [5]:
"""
运行 PSO 优化
"""
# from model.pso import PSO
# pso = PSO(env, num_particles=30, max_iter=100)
# best_position, best_fitness = pso.optimize()

'\n运行 PSO 优化\n'

In [6]:
"""
运行 HS_MOPSO 优化
"""
from model.hs_mopso import HS_MOPSO
mopso = HS_MOPSO(env, max_iter=80, pop_size=120)
mopso.run()

Iter 1, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 2, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 3, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 4, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 5, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 6, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 7, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 8, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 9, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 10, Average Reward: 0.78, Total Distance: 1392.72, Total Time: 789.59, Success Rate: 0.80
Iter 11, Average Reward: 0.78, Total Distance: 1392.72, Tot

In [ ]:
"""
运行 RR 轮询算法
"""
# from model.rr import round_robin_allocation
# round_robin_allocation(env)

In [8]:
"""
运行随机算法
"""
# from model.random import random_allocation
# random_allocation(env)

'\n运行随机算法\n'